### 1. JSON output from dctap

In [4]:
import json
ap = Path('example02.json').read_text(encoding="UTF-8")
for line in ap.splitlines():
  print(line)

{
    "namespaces": {
        "ex": "http://ex.example/#",
        "xsd": "http://www.w3.org/2001/XMLSchema#",
        "school": "http://school.example/#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "shapes": [
        {
            "shapeID": "school:Enrollee",
            "statement_templates": [
                {
                    "min": "1",
                    "max": "2",
                    "propertyID": "ex:hasGuardian",
                    "valueNodeType": "iri"
                },
                {
                    "MinInclusive": "13",
                    "MaxInclusivemax": "20",
                    "propertyID": "foaf:age",
                    "valueDataType": "xsd:integer",
                }
            ],
        }
    ]
}


In [5]:
"""
ShEx template in Jinja2.
"""
shex_jinja = Path('example02.jinja').read_text(encoding="UTF-8")
for line in shex_jinja.splitlines():
   print(line)

shex_jinja = """
{%- for prefix, uri in namespaces.items() %}
PREFIX {{prefix}}: <{{uri}}>
{%- endfor %}
BASE <http://purl.org/yama/example/mybook/v021/#>

{%- for shape in shapes %}
<{{shape.shapeID}}> {
  {%- for statement in shape.statements%}
  {{statement.property}} @<{{statement.statementID}}> { {{statement.minOccur}},{{statement.maxOccur}} };
  {%- endfor %}
}
{%- endfor %}
"""


### 3. Render ShExC Template with Jinja2

In [6]:
from jinja2 import Template
template = Template(shex_jinja)
shexc_output = template.render(ap)
print(shexc_output)

ValueError: dictionary update sequence element #0 has length 1; 2 is required

### 5. Load the PyShExC library

In [5]:
import pyshexc
from pyshexc.parser_impl.generate_shexj import parse

### 6. Convert ShExC to ShExJ

In [6]:
import json
json_ap=parse(shexc_output)._as_json
parsed = json.loads(json_ap)
print(json.dumps(parsed, indent=4, sort_keys=False))

{
    "type": "Schema",
    "shapes": [
        {
            "type": "Shape",
            "id": "http://school.example/#Enrollee",
            "expression": {
                "type": "EachOf",
                "expressions": [
                    {
                        "type": "TripleConstraint",
                        "predicate": "http://ex.example/#hasGuardian",
                        "valueExpr": {
                            "type": "NodeConstraint",
                            "nodeKind": "iri"
                        },
                        "min": 1,
                        "max": 2
                    },
                    {
                        "type": "TripleConstraint",
                        "predicate": "http://xmlns.com/foaf/0.1/age",
                        "valueExpr": {
                            "type": "NodeConstraint",
                            "datatype": "http://www.w3.org/2001/XMLSchema#integer",
                            "mininclusive": 13
    